In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
repository_path='/content/drive/MyDrive/'
folder='DACON/'  #DACON 연구실 이미지 연습
title='서울 랜드마크 분류/서울 분류 데이터/'

In [4]:
full_path=repository_path+folder+title

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.applications import *

Dataset은 샘플과 정답(label)을 저장, DataLoader은 Dataset을 순회 가능한 객체로 감싼다

In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 데이터 다운로드

In [7]:
training_data=datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [8]:
test_data=datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [10]:
, dataloader
batch_size=64

train_dataloader=DataLoader(training_data, batch_size=batch_size)
test_dataloader=DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
    print(X.shape)
    print(y.dtype)

torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64
torch.Size([64, 1, 28, 28])
torch.int64


# 모델 만들기

In [11]:
device='cuda' if torch.cuda.is_available() else "cpu"
print(f'using {device} device')

using cpu device


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X=torch.rand(1,28,28, device=device)
logits=model(X)
pred_probab=nn.Softmax(dim=1)(logits)
y_pred=pred_probab.argmax(1)
print(y_pred)

tensor([4])


## 모델 계층(Layer)

In [15]:
input_image=torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

In [16]:
flatten=nn.Flatten() #연속된 배열 출력
flat_image=flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

선형 계층은 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈

In [17]:
layer1=nn.Linear(in_features=28*28, out_features=20)
hidden1=layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

비선형 활성화 함수는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 형상을 학습할 수 있도록 돕는다. 

In [18]:
print(f'Before ReLU : {hidden1}\n\n')
hidden1=nn.ReLU()(hidden1)
print(f'After ReLU : {hidden1}')

Before ReLU : tensor([[-0.0415, -0.3472, -0.2516,  0.8037,  0.4404,  0.1487,  0.3140, -0.2084,
          0.1419,  0.4907, -0.3692,  0.0121, -0.3167,  0.3089,  0.0594, -0.1469,
         -0.3821, -0.6441, -0.0052, -0.5186],
        [-0.0427, -0.4511,  0.1400,  0.6941,  0.7789,  0.3226,  0.1535, -0.4411,
          0.3949,  0.2651, -0.1885, -0.0375, -0.1703, -0.1008,  0.3061,  0.1003,
         -0.2292, -0.7579, -0.2366, -0.4208],
        [ 0.3301, -0.1053,  0.1633,  0.6930,  0.6977,  0.0590, -0.0256, -0.1289,
         -0.0252, -0.1621, -0.3020, -0.5358, -0.0237,  0.2372, -0.0245,  0.6475,
         -0.1881, -0.5784, -0.1660, -0.6569]], grad_fn=<AddmmBackward0>)


After ReLU : tensor([[0.0000, 0.0000, 0.0000, 0.8037, 0.4404, 0.1487, 0.3140, 0.0000, 0.1419,
         0.4907, 0.0000, 0.0121, 0.0000, 0.3089, 0.0594, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.1400, 0.6941, 0.7789, 0.3226, 0.1535, 0.0000, 0.3949,
         0.2651, 0.0000, 0.0000, 0.0000, 0.0000, 0.

## nn.Sequential

nn.Sequential은 순서를 갖는 모듈의 컨테이너

In [19]:
seq_modules=nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

## nn.Softmax

모델의 각 분류에 대한 예측 확률을 나타내도록 [0,1] 범위로 비례하여 조정

In [20]:
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

# 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화 된다.

In [21]:
print(f'Model structure : {model}\n\n')

for name, param in model.named_parameters():
    print(f'layers : {name} , size {param.size()} , Values : {param[:2]}\n')

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


layers : linear_relu_stack.0.weight , size torch.Size([512, 784]) , Values : tensor([[ 0.0046, -0.0150, -0.0302,  ...,  0.0249,  0.0231, -0.0340],
        [-0.0046,  0.0032,  0.0101,  ...,  0.0284, -0.0348,  0.0256]],
       grad_fn=<SliceBackward0>)

layers : linear_relu_stack.0.bias , size torch.Size([512]) , Values : tensor([ 0.0088, -0.0134], grad_fn=<SliceBackward0>)

layers : linear_relu_stack.2.weight , size torch.Size([512, 512]) , Values : tensor([[ 0.0277, -0.0117, -0.0016,  ...,  0.0161,  0.0359, -0.0243],
        [-0.0106,  0.0370,  0.0207,  ...,  0.0218, -0.0124, -0.0098]],
       grad_fn=<SliceBackward0>)

layers : linear_relu_stack.2.bias

# 모델 최적화하기

## 하이퍼파라미터

In [22]:
learning_rate=1e-3
batch_size=64
epochs=5

## 손실 함수

In [23]:
loss_fn=nn.CrossEntropyLoss()

## 옵티마이저

In [24]:
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

# 전체 구현

In [26]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # 예측과 손실 계산
        pred=model(X)
        loss=loss_fn(pred, y)

        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100==0:
            loss, current=loss.item(), batch*len(X)
            print(f'loss : {loss:>7f} [{current:>5d}/{size:>5d}]')


def test_loop(dataloader, model,loss_fn):
    size=len(dataloader.dataset)
    num_batches=len(dataloader)
    test_loss, correct=0,0

    with torch.no_grad():
        for X,y in dataloader:
            pred=model(X)
            test_loss+=loss_fn(pred, y).item()
            correct+=(pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss/=num_batches
    correct/=size
    print(f'Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f}\n')

In [27]:
epochs=10
for t in range(epochs):
    print(f'Epoch {t+1}\n----------------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('done')

Epoch 1
----------------------------------
loss : 2.302338 [    0/60000]
loss : 2.288900 [ 6400/60000]
loss : 2.269533 [12800/60000]
loss : 2.262230 [19200/60000]
loss : 2.242597 [25600/60000]
loss : 2.217515 [32000/60000]
loss : 2.219711 [38400/60000]
loss : 2.183098 [44800/60000]
loss : 2.175890 [51200/60000]
loss : 2.148586 [57600/60000]
Test Error : 
 Accuracy : 45.9%, Avg loss : 2.144851

Epoch 2
----------------------------------
loss : 2.149447 [    0/60000]
loss : 2.140079 [ 6400/60000]
loss : 2.084855 [12800/60000]
loss : 2.106713 [19200/60000]
loss : 2.038891 [25600/60000]
loss : 1.979673 [32000/60000]
loss : 2.008156 [38400/60000]
loss : 1.920986 [44800/60000]
loss : 1.925528 [51200/60000]
loss : 1.854942 [57600/60000]
Test Error : 
 Accuracy : 54.2%, Avg loss : 1.857393

Epoch 3
----------------------------------
loss : 1.881803 [    0/60000]
loss : 1.853140 [ 6400/60000]
loss : 1.738570 [12800/60000]
loss : 1.793941 [19200/60000]
loss : 1.657976 [25600/60000]
loss : 1.6165